In [ ]:
library("ggplot2")
library("dplyr")
library("viridis")
library("purrr")

# Generate some simulated data

In [ ]:
gen_data <- function(N, P, s = 1 / 10) {
  X <- array(rnorm(N * P), c(N, P))
  beta <- array(rnorm(P + 1), c(P + 1, 1))
  epsilon <- array(rnorm(N, sd = s), c(N, 1))
  y <- cbind(1, X) %*% beta + epsilon
  return(list(
    X = X, beta = beta, epsilon = epsilon, y = y
  ))
}

In [ ]:
d <- gen_data(100, 1)

In [ ]:
plot(d$X, d$y)

In [ ]:
head(d$X)

In [ ]:
head(d$beta)

In [ ]:
head(d$y)

In [ ]:
names(d)

# Fitting in `R` using `lm`

In [ ]:
mod <- lm(d$y ~ d$X)

In [ ]:
summary(mod)

In [ ]:
mod$coef

In [ ]:
head(predict(mod))

In [ ]:
plot(d$y, predict(mod))

# Calculating by hand

In [ ]:
D <- cbind(1, d$X)
head(D)

In [ ]:
beta_hat <- solve(t(D) %*% D) %*% t(D) %*% d$y

In [ ]:
beta_hat

In [ ]:
coef(mod)

# Plotting the loss

In [ ]:
L <- function(beta0, beta1) {
  beta <- c(beta0, beta1)
  beta <- array(beta, c(2, 1))
  sum((d$y - cbind(1, d$X) %*% beta)^2)
}

In [ ]:
beta_grid <- expand.grid(beta0 = seq(-3, 3, by = .1), beta1 = seq(-3, 3, by = .1))

In [ ]:
beta_grid %>% head()

In [ ]:
ggplot(data = beta_grid, mapping = aes(x = beta0, y = beta1)) +
  geom_point()

In [ ]:
beta_grid <- beta_grid %>%
  mutate(lss = pmap_dbl(list(beta0, beta1), L))

In [ ]:
head(beta_grid)

In [ ]:
ggplot(data = beta_grid, mapping = aes(x = beta0, y = beta1, fill = lss, z = lss)) +
  geom_tile() +
  scale_fill_viridis() +
  geom_contour()

# Categorial variables

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/South_Shetland-2016-Deception_Island%E2%80%93Chinstrap_penguin_%28Pygoscelis_antarctica%29_04.jpg/800px-South_Shetland-2016-Deception_Island%E2%80%93Chinstrap_penguin_%28Pygoscelis_antarctica%29_04.jpg" width="200" height="500">

In [ ]:
library("palmerpenguins")

In [ ]:
penguins <- penguins %>% filter(complete.cases(.))

In [ ]:
penguins %>% sample_n(5)

In [ ]:
mod <- lm(flipper_length_mm ~ bill_length_mm + species, data = penguins)

In [ ]:
summary(penguins)

In [ ]:
summary(mod)

In [ ]:
D <- model.matrix(~ bill_length_mm + species, data = penguins)
head(D[sample(nrow(D)), ])

In [ ]:
y <- penguins$flipper_length_mm
y <- array(y, c(length(y), 1))
head(y)

In [ ]:
beta_hat <- solve(t(D) %*% D) %*% t(D) %*% y

In [ ]:
beta_hat

In [ ]:
coef(mod)

# Fitting Issues

In [ ]:
d <- gen_data(100, 200)

In [ ]:
dim(d$X)

In [ ]:
mod <- lm(d$y ~ d$X)
summary(mod)

In [ ]:
tail(coef(mod))

In [ ]:
D <- model.matrix(mod)
D[1:5, 1:5]

In [ ]:
solve(t(D) %*% D) %*% t(D) %*% d$y

another example

In [ ]:
xx <- rnorm(100)
X <- cbind(xx, xx)
colnames(X) <- c("V1", "V2")
head(X)

In [ ]:
true_beta <- array(c(3, 5), c(2, 1))
true_beta

In [ ]:
y <- X %*% true_beta + rnorm(100, sd = 1 / 10)

In [ ]:
mod <- lm(y ~ X)
summary(mod)

In [ ]:
plot(y, predict(mod))

In [ ]:
D <- cbind(1, X)
beta_hat <- solve(t(D) %*% D) %*% t(D) %*% y

what does the loss look like?

In [ ]:
L <- function(beta1, beta2) {
  beta <- c(beta1, beta2)
  beta <- array(beta, c(2, 1))
  sum((y - X %*% beta)^2)
}
beta_grid <- expand.grid(beta1 = seq(-5, 10, by = .1), beta2 = seq(-5, 10, by = .1))
beta_grid <- beta_grid %>%
  mutate(lss = pmap_dbl(list(beta1, beta2), L))

In [ ]:
ggplot(data = beta_grid, mapping = aes(x = beta1, y = beta2, fill = lss, z = lss)) +
  geom_tile() +
  scale_fill_viridis() +
  geom_contour()

# polynomial regression

In [ ]:
N <- 100
P <- 1
X <- array(rnorm(N * P), c(N, P))

In [ ]:
y <- X + 5 * X^2 - X^3 + rnorm(100, 0, .5)
plot(X, y)

In [ ]:
D <- cbind(1, X, X^2, X^3)
head(D)

In [ ]:
beta_hat <- solve(t(D) %*% D) %*% t(D) %*% y

beta_hat

In [ ]:
xp <- seq(-2, 2, length.out = 100)

In [ ]:
Dp <- cbind(1, xp, xp^2, xp^3)

In [ ]:
y_pred <- Dp %*% beta_hat

In [ ]:
plot(X, y)
lines(xp, y_pred, col = "red")